# ResNet18 Baseline Conv2d Benchmark

Сравнение nn.Conv2d и кастомной img2col→GEMM свёртки (Baseline TritonConv2d) на ResNet18 с разными batch size и сценариями спарсификации.


In [1]:
import sys, pathlib
sys.path.insert(0, str(pathlib.Path().resolve().parent))


In [2]:
import copy
import json
import math
import random
from pathlib import Path
from typing import Dict, List, Tuple

import numpy as np
import pandas as pd
import torch
from torch import nn
from torch.utils.data import DataLoader
import torchvision
from torchvision import transforms

from conv_gemm.baseline_layers.triton_conv2d import TritonConv2d


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if device.type != "cuda":
    raise RuntimeError("CUDA GPU is required for this benchmark")

seed = 42
random.seed(seed)
torch.manual_seed(seed)
np.random.seed(seed)
torch.backends.cudnn.benchmark = True

data_root = Path("../data").resolve()
data_root.mkdir(parents=True, exist_ok=True)

config = {
    "data_root": str(data_root),
    "num_classes": 10,
    "batch_sizes": [32, 64, 96, 128, 160, 192, 256],
    "num_workers": 4,
    "train_subset": 8192,
    "lr": 1e-3,
    "momentum": 0.9,
    "weight_decay": 5e-4,
    "warmup_steps": 5,
    "benchmark_steps": 40,
    "baseline_conv": {
        "BLOCK_M": 64,
        "BLOCK_N": 64,
        "BLOCK_K": 64,
        "NUM_WARPS": 4,
        "NUM_STAGES": 2,
    },
    "sparsity_bench": {
        "modes": ["channel", "block", "input"],
        "keep_ratios": [0.75, 0.6, 0.5, 0.25],
        "block_size": 4,
        "batch_size": 128,
    },
}
print(json.dumps(config, indent=2))


{
  "data_root": "/mnt/d/VSCode-Projects/conv2d-img2col-gemm/data",
  "num_classes": 10,
  "batch_sizes": [
    32,
    64,
    96,
    128,
    160,
    192,
    256
  ],
  "num_workers": 4,
  "train_subset": 8192,
  "lr": 0.001,
  "momentum": 0.9,
  "weight_decay": 0.0005,
  "warmup_steps": 5,
  "benchmark_steps": 40,
  "baseline_conv": {
    "BLOCK_M": 64,
    "BLOCK_N": 64,
    "BLOCK_K": 64,
    "NUM_WARPS": 4,
    "NUM_STAGES": 2
  },
  "sparsity_bench": {
    "modes": [
      "channel",
      "block",
      "input"
    ],
    "keep_ratios": [
      0.75,
      0.6,
      0.5,
      0.25
    ],
    "block_size": 4,
    "batch_size": 128
  }
}


In [4]:
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

full_train = torchvision.datasets.CIFAR10(
    root=config["data_root"], train=True, download=True, transform=transform_train
)
if config["train_subset"] is not None and config["train_subset"] < len(full_train):
    g = torch.Generator().manual_seed(seed)
    subset_idx = torch.randperm(len(full_train), generator=g)[: config["train_subset"]]
    train_dataset = torch.utils.data.Subset(full_train, subset_idx)
else:
    train_dataset = full_train


def make_loader(batch_size: int) -> DataLoader:
    return DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,
        drop_last=True,
        num_workers=config["num_workers"],
        pin_memory=True,
    )

train_loaders: Dict[int, DataLoader] = {}
for bs in config["batch_sizes"]:
    train_loaders[bs] = make_loader(bs)

print({bs: len(loader) for bs, loader in train_loaders.items()})


{32: 256, 64: 128, 96: 85, 128: 64, 160: 51, 192: 42, 256: 32}


In [5]:
def make_triton_conv(src: nn.Conv2d, cfg: dict) -> TritonConv2d:
    if src.groups != 1:
        raise ValueError("Baseline TritonConv2d currently supports groups=1 only")
    layer = TritonConv2d(
        in_channels=src.in_channels,
        out_channels=src.out_channels,
        kernel_size=src.kernel_size,
        stride=src.stride,
        padding=src.padding,
        dilation=src.dilation,
        bias=(src.bias is not None),
        **cfg,
    ).to(src.weight.device)
    with torch.no_grad():
        layer.weight.copy_(src.weight.detach().to(layer.weight.dtype))
        if layer.bias is not None and src.bias is not None:
            layer.bias.copy_(src.bias.detach().to(layer.bias.dtype))
    return layer


def replace_convs_with_baseline(module: nn.Module, cfg: dict):
    for name, child in module.named_children():
        if isinstance(child, nn.Conv2d):
            setattr(module, name, make_triton_conv(child, cfg))
        else:
            replace_convs_with_baseline(child, cfg)


def build_model_pair(config: dict):
    reference = torchvision.models.resnet18(num_classes=config["num_classes"])
    baseline = copy.deepcopy(reference)
    replace_convs_with_baseline(baseline, config["baseline_conv"])
    return reference, baseline


def apply_sparsity_to_model(model: nn.Module, mode: str, keep_ratio: float, block_size: int = 4):
    for layer in model.modules():
        if isinstance(layer, TritonConv2d):
            layer.clear_sparsity()
            if keep_ratio >= 1.0:
                continue
            if mode == "channel":
                layer.set_channel_sparsity(keep_ratio)
                layer.set_backward_channel_sparsity(keep_ratio)
            elif mode == "block":
                layer.set_block_sparsity(keep_ratio, block_size=block_size)
                layer.set_backward_block_sparsity(keep_ratio, block_size=block_size)
            elif mode == "input":
                layer.set_input_channel_sparsity(keep_ratio)
                layer.set_backward_input_channel_sparsity(keep_ratio)
            else:
                raise ValueError(f"Unknown sparsity mode: {mode}")


In [6]:
def run_benchmark(model: nn.Module, label: str, loader: DataLoader, config: dict):
    model = model.to(device)
    model.train()
    criterion = nn.CrossEntropyLoss().to(device)
    optimizer = torch.optim.SGD(
        model.parameters(),
        lr=config["lr"],
        momentum=config["momentum"],
        weight_decay=config["weight_decay"],
    )

    warmup = config["warmup_steps"]
    total_steps = config["benchmark_steps"]
    records = []
    data_iter = iter(loader)

    for step in range(total_steps):
        try:
            images, targets = next(data_iter)
        except StopIteration:
            data_iter = iter(loader)
            images, targets = next(data_iter)

        images = images.to(device, non_blocking=True)
        targets = targets.to(device, non_blocking=True)

        optimizer.zero_grad(set_to_none=True)
        torch.cuda.reset_peak_memory_stats(device)
        torch.cuda.synchronize()

        fwd_start = torch.cuda.Event(enable_timing=True)
        fwd_end = torch.cuda.Event(enable_timing=True)
        bwd_start = torch.cuda.Event(enable_timing=True)
        bwd_end = torch.cuda.Event(enable_timing=True)

        fwd_start.record()
        outputs = model(images)
        fwd_end.record()
        loss = criterion(outputs, targets)

        bwd_start.record()
        loss.backward()
        bwd_end.record()
        optimizer.step()

        torch.cuda.synchronize()

        fwd_ms = fwd_start.elapsed_time(fwd_end)
        bwd_ms = bwd_start.elapsed_time(bwd_end)
        step_ms = fwd_ms + bwd_ms
        mem_alloc = torch.cuda.max_memory_allocated(device) / 1024 ** 2
        mem_reserved = torch.cuda.max_memory_reserved(device) / 1024 ** 2

        if step >= warmup:
            records.append({
                "label": label,
                "step": step,
                "loss": float(loss.item()),
                "fwd_ms": fwd_ms,
                "bwd_ms": bwd_ms,
                "step_ms": step_ms,
                "throughput_sps": images.size(0) / (step_ms / 1000.0),
                "max_mem_alloc_mb": mem_alloc,
                "max_mem_reserved_mb": mem_reserved,
            })

    if not records:
        raise RuntimeError("No data recorded for benchmark")

    df = pd.DataFrame(records)
    summary = {
        "label": label,
        "avg_forward_ms": df["fwd_ms"].mean(),
        "avg_backward_ms": df["bwd_ms"].mean(),
        "avg_step_ms": df["step_ms"].mean(),
        "samples_per_s": df["throughput_sps"].mean(),
        "max_mem_alloc_mb": df["max_mem_alloc_mb"].max(),
        "max_mem_reserved_mb": df["max_mem_reserved_mb"].max(),
    }
    return df, summary


Таблица `summary_df` показывает средние метрики по каждому batch size: `avg_forward_ms`, `avg_backward_ms`, `avg_step_ms`, `samples_per_s`, а также пики памяти (`max_mem_alloc_mb`, `max_mem_reserved_mb`).


In [7]:
batch_summaries = []
batch_details = []

for bs, loader in train_loaders.items():
    print(f"=== Batch size {bs} ===")
    torch_model, baseline_model = build_model_pair(config)

    torch_df, torch_summary = run_benchmark(torch_model, f"nn.Conv2d (bs={bs})", loader, config)
    torch_summary.update({"variant": "nn.Conv2d", "batch_size": bs})
    batch_summaries.append(torch_summary)
    batch_details.append(torch_df.assign(variant="nn.Conv2d", batch_size=bs))

    baseline_df, baseline_summary = run_benchmark(baseline_model, f"Baseline TritonConv2d (bs={bs})", loader, config)
    baseline_summary.update({"variant": "Baseline TritonConv2d", "batch_size": bs})
    batch_summaries.append(baseline_summary)
    batch_details.append(baseline_df.assign(variant="Baseline TritonConv2d", batch_size=bs))

summary_df = pd.DataFrame(batch_summaries).set_index(["variant", "batch_size"])
summary_df


=== Batch size 32 ===
=== Batch size 64 ===
=== Batch size 96 ===
=== Batch size 128 ===
=== Batch size 160 ===
=== Batch size 192 ===
=== Batch size 256 ===


,,label,avg_forward_ms,avg_backward_ms,avg_step_ms,samples_per_s,max_mem_alloc_mb,max_mem_reserved_mb
variant,batch_size,,,,,,,
nn.Conv2d,32,nn.Conv2d (bs=32),6.945951,7.504282,14.450233,2321.329362,192.057129,220.0
Baseline TritonConv2d,32,Baseline TritonConv2d (bs=32),45.697910,18.743004,64.440913,498.520284,207.492676,226.0
nn.Conv2d,64,nn.Conv2d (bs=64),7.130069,6.846434,13.976503,4738.278499,190.058105,228.0
Baseline TritonConv2d,64,Baseline TritonConv2d (bs=64),46.524829,19.813932,66.338762,973.842799,228.024902,258.0
nn.Conv2d,96,nn.Conv2d (bs=96),5.744413,8.117101,13.861514,7074.106999,190.685059,222.0
Baseline TritonConv2d,96,Baseline TritonConv2d (bs=96),44.192646,22.477297,66.669944,1448.606724,252.019043,312.0
nn.Conv2d,128,nn.Conv2d (bs=128),8.705702,8.687528,17.393231,7575.380323,204.617676,228.0
Baseline TritonConv2d,128,Baseline TritonConv2d (bs=128),46.460595,27.343490,73.804085,1750.624751,281.832520,340.0
nn.Conv2d,160,nn.Conv2d (bs=160),8.192318,13.367062,21.559380,7779.820179,219.119629,252.0


Вывод `detail_df.groupby(...).describe()` содержит count/mean/std/min/25%/50%/75%/max для метрик `step_ms`, `fwd_ms`, `bwd_ms`, `max_mem_alloc_mb` отдельно по каждому `(variant, batch_size)`.


In [8]:
detail_df = pd.concat(batch_details, ignore_index=True)
metrics = ["step_ms", "fwd_ms", "bwd_ms", "max_mem_alloc_mb"]
detail_df.groupby(["variant", "batch_size"])[metrics].describe()


step_ms                                  \
                                   count       mean       std        min   
variant               batch_size                                           
Baseline TritonConv2d 32            35.0  64.440913  4.251413  56.388992   
                      64            35.0  66.338762  6.978582  58.422527   
                      96            35.0  66.669944  5.457417  60.666687   
                      128           35.0  73.804085  7.826183  65.466335   
                      160           35.0  78.921672  7.717207  68.343361   
                      192           35.0  79.731534  6.119224  73.734718   
                      256           35.0  97.125984  6.911990  89.954685   
nn.Conv2d             32            35.0  14.450233  3.935573  11.075040   
                      64            35.0  13.976503  2.805161  10.746112   
                      96            35.0  13.861514  2.371935  12.315904   
                      128           35.0  17.393231  3.226433  13.574464   
                      160           35.0  21.559380  4.983187  15.366944   
                      192           35.0  18.188941  2.140933  16.097088   
                      256           35.0  22.381140  3.178098  19.657696   

                                                                    \
                                        25%        50%         75%   
variant               batch_size                                     
Baseline TritonConv2d 32          62.039184  63.634241   65.847728   
                      64          62.235024  64.566689   68.469855   
                      96          62.735361  64.800766   69.165567   
                      128         68.828208  71.731010   74.427024   
                      160         73.366129  77.659521   81.677439   
                      192         75.315536  77.947838   80.613407   
                      256         91.713455  95.792736  100.641424   
nn.Conv2d             32          12.327680  13.103872   14.478848   
                      64          11.784752  12.721408   15.659072   
                      96          12.564736  12.817376   13.792912   
                      128         15.503536  16.532577   18.523696   
                      160         17.359216  20.084896   26.242592   
                      192         16.856624  17.436352   18.659232   
                      256         20.026768  21.066752   23.349056   

                                             fwd_ms             ...  \
                                         max  count       mean  ...   
variant               batch_size                                ...   
Baseline TritonConv2d 32           80.410431   35.0  45.697910  ...   
                      64           86.814560   35.0  46.524829  ...   
                      96           81.938688   35.0  44.192646  ...   
                      128         102.152164   35.0  46.460595  ...   
                      160         101.251940   35.0  45.952003  ...   
                      192          96.305569   35.0  42.599159  ...   
                      256         123.397758   35.0  44.108297  ...   
nn.Conv2d             32           30.395841   35.0   6.945951  ...   
                      64           21.054720   35.0   7.130069  ...   
                      96           23.297791   35.0   5.744413  ...   
                      128          26.371520   35.0   8.705702  ...   
                      160          31.677920   35.0   8.192318  ...   
                      192          26.378241   35.0   6.524090  ...   
                      256          30.037889   35.0   7.873459  ...   

                                     bwd_ms            max_mem_alloc_mb  \
                                        75%        max            count   
variant               batch_size                                          
Baseline TritonConv2d 32          18.828288  28.275711             35.0   
                      64          20.2833

`baseline_vs_torch_df` сравнивает nn.Conv2d и Baseline TritonConv2d: пары столбцов с абсолютными значениями (forward/backward/step время, throughput, память) и коэффициенты ускорения (`speedup_*`, `throughput_ratio`, `mem_*_ratio`).


In [9]:
baseline_compare_rows = []
for bs in config["batch_sizes"]:
    torch_row = summary_df.loc[("nn.Conv2d", bs)]
    baseline_row = summary_df.loc[("Baseline TritonConv2d", bs)]
    comparison = {
        "batch_size": bs,
        "torch_forward_ms": torch_row["avg_forward_ms"],
        "baseline_forward_ms": baseline_row["avg_forward_ms"],
        "torch_backward_ms": torch_row["avg_backward_ms"],
        "baseline_backward_ms": baseline_row["avg_backward_ms"],
        "torch_step_ms": torch_row["avg_step_ms"],
        "baseline_step_ms": baseline_row["avg_step_ms"],
        "torch_samples_per_s": torch_row["samples_per_s"],
        "baseline_samples_per_s": baseline_row["samples_per_s"],
        "speedup_forward": torch_row["avg_forward_ms"] / baseline_row["avg_forward_ms"],
        "speedup_backward": torch_row["avg_backward_ms"] / baseline_row["avg_backward_ms"],
        "speedup_step": torch_row["avg_step_ms"] / baseline_row["avg_step_ms"],
        "throughput_ratio": baseline_row["samples_per_s"] / torch_row["samples_per_s"],
        "torch_mem_alloc_mb": torch_row["max_mem_alloc_mb"],
        "baseline_mem_alloc_mb": baseline_row["max_mem_alloc_mb"],
        "torch_mem_reserved_mb": torch_row["max_mem_reserved_mb"],
        "baseline_mem_reserved_mb": baseline_row["max_mem_reserved_mb"],
        "mem_alloc_ratio": baseline_row["max_mem_alloc_mb"] / torch_row["max_mem_alloc_mb"],
        "mem_reserved_ratio": baseline_row["max_mem_reserved_mb"] / torch_row["max_mem_reserved_mb"],
    }
    baseline_compare_rows.append(comparison)

baseline_vs_torch_df = pd.DataFrame(baseline_compare_rows).set_index("batch_size")
baseline_vs_torch_df


,torch_forward_ms,baseline_forward_ms,torch_backward_ms,baseline_backward_ms,torch_step_ms,baseline_step_ms,torch_samples_per_s,baseline_samples_per_s,speedup_forward,speedup_backward,speedup_step,throughput_ratio,torch_mem_alloc_mb,baseline_mem_alloc_mb,torch_mem_reserved_mb,baseline_mem_reserved_mb,mem_alloc_ratio,mem_reserved_ratio
batch_size,,,,,,,,,,,,,,,,,,
32,6.945951,45.697910,7.504282,18.743004,14.450233,64.440913,2321.329362,498.520284,0.151997,0.400378,0.224240,0.214756,192.057129,207.492676,220.0,226.0,1.080370,1.027273
64,7.130069,46.524829,6.846434,19.813932,13.976503,66.338762,4738.278499,973.842799,0.153253,0.345536,0.210684,0.205527,190.058105,228.024902,228.0,258.0,1.199764,1.131579
96,5.744413,44.192646,8.117101,22.477297,13.861514,66.669944,7074.106999,1448.606724,0.129986,0.361124,0.207912,0.204776,190.685059,252.019043,222.0,312.0,1.321651,1.405405
128,8.705702,46.460595,8.687528,27.343490,17.393231,73.804085,7575.380323,1750.624751,0.187378,0.317718,0.235668,0.231094,204.617676,281.832520,228.0,340.0,1.377362,1.491228
160,8.192318,45.952003,13.367062,32.969670,21.559380,78.921672,7779.820179,2044.275960,0.178280,0.405435,0.273174,0.262766,219.119629,306.271973,252.0,366.0,1.397739,1.452381
192,6.524090,42.599159,11.664851,37.132375,18.188941,79.731534,10674.234165,2420.601214,0.153151,0.314142,0.228127,0.226770,230.495605,334.960449,274.0,402.0,1.453218,1.467153
256,7.873459,44.108297,14.507681,53.017687,22.381140,97.125984,11634.171224,2647.492920,0.178503,0.273639,0.230434,0.227562,258.373535,383.338379,306.0,598.0,1.483660,1.954248


Таблица `summary_df` показывает средние метрики по каждому batch size: `avg_forward_ms`, `avg_backward_ms`, `avg_step_ms`, `samples_per_s`, а также пики памяти (`max_mem_alloc_mb`, `max_mem_reserved_mb`).


In [10]:
sparsity_cfg = config["sparsity_bench"]
sparsity_bs = sparsity_cfg["batch_size"]
if sparsity_bs not in train_loaders:
    train_loaders[sparsity_bs] = make_loader(sparsity_bs)
sparsity_loader = train_loaders[sparsity_bs]

sparsity_summaries = []
sparsity_details = []

for mode in sparsity_cfg["modes"]:
    for ratio in sparsity_cfg["keep_ratios"]:
        _, baseline_model = build_model_pair(config)
        apply_sparsity_to_model(
            baseline_model,
            mode,
            keep_ratio=ratio,
            block_size=sparsity_cfg.get("block_size", 4),
        )
        label = f"{mode.capitalize()} sparsity (keep={ratio:.2f}, bs={sparsity_bs})"
        bench_df, bench_summary = run_benchmark(baseline_model, label, sparsity_loader, config)
        bench_summary.update({
            "variant": f"Sparsity::{mode}",
            "mode": mode,
            "keep_ratio": ratio,
            "batch_size": sparsity_bs,
        })
        sparsity_summaries.append(bench_summary)
        sparsity_details.append(
            bench_df.assign(variant=f"Sparsity::{mode}", mode=mode, keep_ratio=ratio, batch_size=sparsity_bs)
        )

sparsity_summary_df = pd.DataFrame(sparsity_summaries).sort_values("samples_per_s", ascending=False).reset_index(drop=True)
sparsity_summary_df


,label,avg_forward_ms,avg_backward_ms,avg_step_ms,samples_per_s,max_mem_alloc_mb,max_mem_reserved_mb,variant,mode,keep_ratio,batch_size
0,"Channel sparsity (keep=0.75, bs=128)",58.553237,27.390333,85.943570,1495.724046,281.906738,602.0,Sparsity::channel,channel,0.75,128
1,"Block sparsity (keep=0.50, bs=128)",58.612525,27.804467,86.416992,1494.505578,277.916504,602.0,Sparsity::block,block,0.50,128
2,"Channel sparsity (keep=0.60, bs=128)",58.730010,28.239315,86.969325,1479.867645,280.566895,602.0,Sparsity::channel,channel,0.60,128
3,"Block sparsity (keep=0.75, bs=128)",60.174384,27.837381,88.011765,1464.490586,281.906738,602.0,Sparsity::block,block,0.75,128
4,"Channel sparsity (keep=0.50, bs=128)",62.127724,27.842911,89.970635,1441.579470,277.916504,602.0,Sparsity::channel,channel,0.50,128
5,"Channel sparsity (keep=0.25, bs=128)",62.129142,27.794782,89.923925,1435.233769,276.178711,602.0,Sparsity::channel,channel,0.25,128
6,"Input sparsity (keep=0.50, bs=128)",60.846281,29.797142,90.643423,1420.362348,257.366699,602.0,Sparsity::input,input,0.50,128
7,"Block sparsity (keep=0.60, bs=128)",61.405683,29.465658,90.871340,1419.049735,280.562500,602.0,Sparsity::block,block,0.60,128
8,"Input sparsity (keep=0.60, bs=128)",70.232736,28.134634,98.367370,1416.643851,260.319336,602.0,Sparsity::input,input,0.60,128
9,"Input sparsity (keep=0.75, bs=128)",61.056020,31.047008,92.103029,1404.039844,267.901855,602.0,Sparsity::input,input,0.75,128


`sparsity_compare_df` добавляет к тем же сценариям относительные значения относительно эталонного nn.Conv2d (`speedup_*_vs_torch`, `throughput_ratio_vs_torch`, `mem_*_ratio_vs_torch`).


In [11]:
sparsity_reference = summary_df.loc[("nn.Conv2d", sparsity_bs)]

sparsity_compare_df = sparsity_summary_df.copy()
sparsity_compare_df["speedup_forward_vs_torch"] = sparsity_reference["avg_forward_ms"] / sparsity_compare_df["avg_forward_ms"]
sparsity_compare_df["speedup_backward_vs_torch"] = sparsity_reference["avg_backward_ms"] / sparsity_compare_df["avg_backward_ms"]
sparsity_compare_df["speedup_step_vs_torch"] = sparsity_reference["avg_step_ms"] / sparsity_compare_df["avg_step_ms"]
sparsity_compare_df["throughput_ratio_vs_torch"] = sparsity_compare_df["samples_per_s"] / sparsity_reference["samples_per_s"]
sparsity_compare_df["mem_alloc_ratio_vs_torch"] = sparsity_compare_df["max_mem_alloc_mb"] / sparsity_reference["max_mem_alloc_mb"]
sparsity_compare_df["mem_reserved_ratio_vs_torch"] = sparsity_compare_df["max_mem_reserved_mb"] / sparsity_reference["max_mem_reserved_mb"]
sparsity_compare_df = sparsity_compare_df.sort_values("samples_per_s", ascending=False).reset_index(drop=True)
sparsity_compare_df


,label,avg_forward_ms,avg_backward_ms,avg_step_ms,samples_per_s,max_mem_alloc_mb,max_mem_reserved_mb,variant,mode,keep_ratio,batch_size,speedup_forward_vs_torch,speedup_backward_vs_torch,speedup_step_vs_torch,throughput_ratio_vs_torch,mem_alloc_ratio_vs_torch,mem_reserved_ratio_vs_torch
0,"Channel sparsity (keep=0.75, bs=128)",58.553237,27.390333,85.943570,1495.724046,281.906738,602.0,Sparsity::channel,channel,0.75,128,0.148680,0.317175,0.202380,0.197445,1.377724,2.640351
1,"Block sparsity (keep=0.50, bs=128)",58.612525,27.804467,86.416992,1494.505578,277.916504,602.0,Sparsity::block,block,0.50,128,0.148530,0.312451,0.201271,0.197285,1.358223,2.640351
2,"Channel sparsity (keep=0.60, bs=128)",58.730010,28.239315,86.969325,1479.867645,280.566895,602.0,Sparsity::channel,channel,0.60,128,0.148233,0.307639,0.199993,0.195352,1.371176,2.640351
3,"Block sparsity (keep=0.75, bs=128)",60.174384,27.837381,88.011765,1464.490586,281.906738,602.0,Sparsity::block,block,0.75,128,0.144675,0.312081,0.197624,0.193322,1.377724,2.640351
4,"Channel sparsity (keep=0.50, bs=128)",62.127724,27.842911,89.970635,1441.579470,277.916504,602.0,Sparsity::channel,channel,0.50,128,0.140126,0.312019,0.193321,0.190298,1.358223,2.640351
5,"Channel sparsity (keep=0.25, bs=128)",62.129142,27.794782,89.923925,1435.233769,276.178711,602.0,Sparsity::channel,channel,0.25,128,0.140123,0.312560,0.193422,0.189460,1.349730,2.640351
6,"Input sparsity (keep=0.50, bs=128)",60.846281,29.797142,90.643423,1420.362348,257.366699,602.0,Sparsity::input,input,0.50,128,0.143077,0.291556,0.191886,0.187497,1.257793,2.640351
7,"Block sparsity (keep=0.60, bs=128)",61.405683,29.465658,90.871340,1419.049735,280.562500,602.0,Sparsity::block,block,0.60,128,0.141774,0.294836,0.191405,0.187324,1.371155,2.640351
8,"Input sparsity (keep=0.60, bs=128)",70.232736,28.134634,98.367370,1416.643851,260.319336,602.0,Sparsity::input,input,0.60,128,0.123955,0.308784,0.176819,0.187006,1.272223,2.640351
9,"Input sparsity (keep=0.75, bs=128)",61.056020,31.047008,92.103029,1404.039844,267.901855,602.0,Sparsity::input,input,0.75,128,0.142585,0.279819,0.188845,0.185342,1.309280,2.640351


`ranking_df` — упорядоченный рейтинг сценариев спарсификации: показывает `mode`, `keep_ratio`, абсолютный throughput и его отношение к торчу, а также ускорения forward/backward/step и изменение памяти.


In [12]:
ranking_df = sparsity_compare_df[[
    "variant",
    "mode",
    "keep_ratio",
    "samples_per_s",
    "throughput_ratio_vs_torch",
    "speedup_forward_vs_torch",
    "speedup_backward_vs_torch",
    "speedup_step_vs_torch",
    "mem_alloc_ratio_vs_torch",
    "mem_reserved_ratio_vs_torch",
]].copy()
ranking_df = ranking_df.sort_values("throughput_ratio_vs_torch", ascending=False).reset_index(drop=True)
ranking_df


,variant,mode,keep_ratio,samples_per_s,throughput_ratio_vs_torch,speedup_forward_vs_torch,speedup_backward_vs_torch,speedup_step_vs_torch,mem_alloc_ratio_vs_torch,mem_reserved_ratio_vs_torch
0,Sparsity::channel,channel,0.75,1495.724046,0.197445,0.148680,0.317175,0.202380,1.377724,2.640351
1,Sparsity::block,block,0.50,1494.505578,0.197285,0.148530,0.312451,0.201271,1.358223,2.640351
2,Sparsity::channel,channel,0.60,1479.867645,0.195352,0.148233,0.307639,0.199993,1.371176,2.640351
3,Sparsity::block,block,0.75,1464.490586,0.193322,0.144675,0.312081,0.197624,1.377724,2.640351
4,Sparsity::channel,channel,0.50,1441.579470,0.190298,0.140126,0.312019,0.193321,1.358223,2.640351
5,Sparsity::channel,channel,0.25,1435.233769,0.189460,0.140123,0.312560,0.193422,1.349730,2.640351
6,Sparsity::input,input,0.50,1420.362348,0.187497,0.143077,0.291556,0.191886,1.257793,2.640351
7,Sparsity::block,block,0.60,1419.049735,0.187324,0.141774,0.294836,0.191405,1.371155,2.640351
8,Sparsity::input,input,0.60,1416.643851,0.187006,0.123955,0.308784,0.176819,1.272223,2.640351
9,Sparsity::input,input,0.75,1404.039844,0.185342,0.142585,0.279819,0.188845,1.309280,2.640351
